# Programación Entera: Heurísticas para el Problema de Localización de Servicios

## Descripción

En este trabajo se estudia el problema de localización de servicios de manera detallada, en concreto el localización de costos fijos, analizando el problema de programación lineal además de la heurística *ADD*. Todo ello se ha hecho apoyandose en el lenguaje *R* y el solver *glpk*. Por último se ha comprobado la validez de los métodos sobre distintos problemas a modo de ejemplo.

Este documento forma parte de un grupo de trabajos relacionados con la heuristicas para la resolución de problemas de programación entera. Existen otros tres trabajos referidos al estudio del [problema de la asignación](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-assignment-heuristics.ipynb), al [problema de la mochila](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-knapsack-heuristics.ipynb) y a [problemas de redes](https://nbviewer.jupyter.org/github/garciparedes/integer-programming-heuristics/blob/master/integer-programming-network-heuristics.ipynb). El contenido completo del trabajo está publicado en el siguiente repositorio: https://github.com/garciparedes/integer-programming-heuristics
## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Introducción](#Introducción)
  * [Problemas de Localización de Servicios](#Problemas-de-Localización-de-Servicios)
    * [Localización con Costos Fijos](#Localización-con-Costos-Fijos)
  * [Conclusiones](#Conclusiones)

## Introducción

Los problemas de localización de servicios son una clase de problemas de programación lineal mixta (variables enteras y continuas) que permite la resolución de una gran cantidad de problemas de optimización que se dan en nuestra sociedad. A grandes rasgos, estos modelos permiten el estudio de aquellos problemas de decisión basados en la búsqueda del lugar óptimo donde colocar un servicio, de tal manera que se minimice una determinada métrica. Es por ello que su ámbito de aplicación es tan extenso. 

El caso prototípico es aquel en que una determinada empresa va a llevar a cabo una ampliación de sucursales. Es por ello que necesita seleccionar la alternativa que mayor beneficio le proporcione. Para ello, puede tratar de definir el problema de distintas maneras, como la de cubrir la zona en que se encuentran todos sus clientes, o más modestamente, una proporción de la misma, así como reducir la distancia según función de error u otras sitituaciones. Las más habituales son las siguientes:

  * Cubrimiento Total (Set Covering). 
  * Cubrimiento Máximo.
  * Localización de Medianas.
  * Localización de Centros de Emergencia. 
  * Localización con Costos Fijos. 
  * Distritos.
  
En este trabajo, vamos a centrarnos en el problema de *Localización con Costos Fijos*, el cual se basa en la búsqueda de la configuración óptima entre un conjunto de puntos de demanda y un conjunto de puntos de abastecimiento posibles, entre los cuales se debe escoger un subconjunto de los mismos para satisfacer las necesidades de los puntos de demanda. 

El resto del trabajo se distribuye de la siguiente manera: Inicialmente se han definido distintas constantes y funciones que se utilizarán en el resto de apartados del trabajo. Posteriormente, se describirá más en detalle el modelo de los problemas de localización de servicios, para después centrarnos en el caso de *Localización de Costes Fijos*, describiendo el modelo en detalle y analizando la heurística *ADD* (muy usada en entornos reales). Después se resolverá el problema *WebBook* describiendo paso a paso las restricciones que vamos añadiendo al problema. Finalmente se discuten distintos aspectos de este tipo de problemas en el apartado de conclusiones.

#### Configuración del Entorno

In [1]:
rm(list = ls())

In [2]:
library(slam, warn.conflicts = FALSE)
library(Rglpk, warn.conflicts = FALSE)
library(purrr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(ggthemes, warn.conflicts = FALSE)
library(scales, warn.conflicts = FALSE)

Using the GLPK callable library version 4.65


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 600)

#### Constantes y Funciones de Apoyo

In [4]:
Solve <- function(...) {
    Rglpk_solve_LP(...)
}

In [5]:
FormatUncapacitedFacicilityLocationSolution <- function(s, p) {
    list(
        min.cost = s$optimum,
        ratios = matrix(round(s$solution[1:(p$m * p$n)], digits = 3), nrow = p$n),
        services = matrix(s$solution[(p$m * p$n + 1):(p$n * (1 + p$m))], nrow = 1),
        amount = round(t(t(matrix(s$solution[1:(p$m * p$n)], nrow = p$n)) * p$d), digits = 3)
    )
}

## Problemas de Localización de Servicios

Tal y como se ha indicado anteriormente, los problema de localización de servicios tratan de resolver el problema de buscar el lugar óptimo en que colocar puntos de abastecimiento de entre un conjunto de posibles puntos, para cumplir las necesidades de un conjunto de puntos de demanda, tratando de reducir al máximo los costes.

### Localización con Costos Fijos

#### Descripción

El problema de localización con costos fijos se basa en lo siguiente: suponemos que tenemos un conjunto de $m$ puntos de demanda, los cuales tienen una determinada cantidad de demanda denotada por $d_i$ (para representar la demanda del punto $i$-ésimo). Además, existe un conjunto de $n$ posibles puntos de abastecimiento, los cuales se definen por una tupla $(f_j, s_j)$, que representa el coste de fijo de utilización, y la capacidad máxima para el punto $j$-ésimo. Entonces, el problema se basa en decidir la planificación óptima entre puntos de demanda y abastecimiento, teniendo en cuenta los costes fijos de utilización. Además, existe una matriz $C$ de $m$ filas y $n$ columnas, que almacena los costos de utilización del punto de abastecimiento $j$-ésimo por el punto de demanda $i$-ésimo.

Este problema se puede modelizar de distintas maneras, sin embargo una de las más utilizadas utiliza las siguientes variables de decisión: Un vector $X$ de longitud $n$ y naturaleza binaria que indicará si es o no utilizado un determinado punto de abastecimiento (lo cual se utilizará para contabilidar los costos fijos), junto con una matriz $Y$ de $m$ filas y $n$ columnas que de naturaleza continua, que indica el ratio de utilización del punto de abastecimiento $j$-ésimo por el punto de demanda $i$-ésimo. Por tanto, estas variables tomarán valores únicamente en el intervalo $[0, 1]$.

#### Modelo

A continuación se incluye la modelización como problema de programación lineal para el problema de localización con costos fijos. Para ellos, nos hemos apoyado en la notación descrita en el anterior apartado.

\begin{equation}
    \begin{array}{ll@{}ll}
      \text{Minimizar} & \displaystyle \sum\limits_{j=1}^{n} f_{j} \cdot x_{j} + \sum\limits_{i=1}^{m}\sum\limits_{j=1}^{n} c_{ij} \cdot d_{i} \cdot y_{ij} \\
      \text{sujeto a}  & \sum\limits_{j=1}^{n} y_{ij} = 1, & \forall i \in \{1,...,m\}\\
                       & \sum\limits_{i=1}^{m} d_{i} \cdot y_{ij} - s_j \cdot x_{j} \leq 0, & \forall j \in \{1,...,n\}\\
                       & y_{ij} \geq 0, & \forall i \in \{1,...,m\}, \forall j \in \{1,...,n\} \\
                       & x_{j} \in \{0, 1\}, & \forall j \in \{1,...,n\}
    \end{array}
\end{equation}

In [6]:
SolveUncapacitedFacicilityLocationExact <- function(p, filtered = NULL) {
    
    conditions <- matrix(0, nrow = p$n + p$m, ncol = (1 + p$m) * p$n)

    for (i in 1:p$m) {
        conditions[i, seq(p$n * (i - 1) + 1, length.out = p$n)] <- 1
    }

    for (i in 1:p$n) {
        conditions[p$m + i, seq(i, length.out = p$m, by = p$n)] <- p$d
        if (is.null(filtered) || i %in% filtered) {
            conditions[p$m + i, p$m * p$n + i] <- -p$s[i]   
        }
    }

    rhs <- rep(c(1, 0), c(p$m, p$n))
    obj <- c(t(t(p$c) * p$d), p$f)

    dir <- rep(c("==", "<="), c(p$m, p$n))
    types <- rep(c("C", "B"), c(p$m, 1) * p$n)

    s <- Solve(obj = obj, 
               mat = conditions, 
               dir = dir, 
               rhs = rhs,
               types = types)
    
    return(s)
}


#### Heurísticas

Para tratar de resolver el problema de costos fijos utilizando un menor coste computacional, existen distintas heurísticas que proporcionan resultados razonables utilizando un menor coste computacional. En este caso, se va a estudiar la heurística ADD para el problema de *localización de servicios con costos fijos*.

##### ADD

TODO(@garciparedes): Añadir Descripción de la Heurística ADD.

In [7]:
SolveUncapacitedFacicilityLocationADD <- function(p, ...) {
    stop("SolveUncapacitedFacicilityLocationADD() unimplemented yet!")
}

#### Ejemplo: WebBook

Web Book vende libros por internet. La administración quiere saber en qué sitios se deben ubicar almacenes de distribución, de entre $5$ sitios posibles, que denotaremos por *A1*, *A2*, *A3*, *A4* y *A5*. La región de ventas está dividida en $5$ zonas indicadas por *NO*, *SO*, *MO*, *SE* y *NE*. Se proporcionan los siguientes datos: la demanda semanal promedio de cada región (en número de libros), el costo promedio de envío de un libro (Euros/libro), el costo fijo semanal (en Euros) si el almacén está en funcionamiento, y la capacidad de cada almacén (en número de libros).

Los datos necesarios para la resolución del problema se definen a continuación, siendo `n` el número de puntos de demanda, `m` el de posibles puntos de abastecimiento, `c` la matriz de costes, `f` los costes fijos, `s` las capacidades de cada punto de abastecimiento y `d` la demanda de cada punto de demanda.

In [8]:
p <- list(n = 5,
          m = 5,
          c = matrix(c(2.40, 3.50, 4.80, 6.80, 5.75,
                       3.25, 2.30, 3.40, 5.25, 6.00,
                       4.05, 3.25, 2.85, 4.30, 4.75,
                       5.25, 6.05, 4.30, 3.25, 2.75,
                       6.95, 5.85, 4.80, 2.10, 3.50), 
                     nrow = 5, byrow = TRUE),
          f = c(4000, 3000, 2500, 4000, 3000),
          s = c(20000, 20000, 15000, 25000, 15000),
          d = c(8000, 12000, 9000, 14000, 17000)
)

##### WebBook: Cuestión 1

En este apartado se resuelve el problema de manera exacta, proporcionando los siguientes resultados (nótese que la solución obtenida para la matriz $Y$ (o de ratios) es binaria)

In [9]:
s <- SolveUncapacitedFacicilityLocationExact(p)
FormatUncapacitedFacicilityLocationSolution(s, p)

1,0,0,0,0
0,1,0,0,0
0,0,1,0,0
0,0,0,0,1
0,0,0,1,0
1,1,1,1,1
8000,0,0,0,0
0,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


##### WebBook: Cuestión 2

*"Un directivo de la compañía piensa que la mejor opción es abrir los almacenes *A1*, *A2* y *A4*. Para esta opción, calcular manualmente su correspodiente costo."*

En este caso, se restringe el espacio de posibles soluciones a las que se basen en los almacenes 1, 2 y 4. La solución obtenida en este caso es la siguiente (nótese que en este caso las soluciones obtenidas en la matriz $Y$ (o de ratios) no son binarias).

In [10]:
s <- SolveUncapacitedFacicilityLocationExact(p, c(1, 2, 4))
FormatUncapacitedFacicilityLocationSolution(s, p)

1,0.083,0,0,0.353
0,0.917,1,0,0.000
0,0.000,0,0,0.000
0,0.000,0,1,0.647
0,0.000,0,0,0.000
1,1,0,1,0
8000,1000,0,0,6000
0,11000,9000,0,0
0,0,0,0,0
0,0,0,14000,11000
0,0,0,0,0


##### WebBook: Cuestión 3

*"Añadir las siguientes restricciones:*
  * *Por proximidad geográfica, los almacenes *A1* y *A3* no deberían estar abiertos simultáneamente.*
  * *Para mantener un equilibrio entre los distribuidores, si al almacén *A4* se le asignan más de $22.000$ libros semanales, los almacenes *A1* y *A2* deben tener asignados, en conjunto, al menos $15.000$ libros.*"
  
En este caso, vamos a resolver el problema paso a paso, construyendo de manera apropiada la matriz de restricciones. A continuación se construye la matriz de condiciones inicial:

In [11]:
conditions <- matrix(0, nrow = p$n + p$m, ncol = (1 + p$m) * p$n)
for (i in 1:p$m) {
    conditions[i, seq(p$n * (i - 1) + 1, length.out = p$n)] <- 1
}
for (i in 1:p$n) {
    conditions[p$m + i, seq(i, length.out = p$m, by = p$n)] <- p$d
    conditions[p$m + i, p$m * p$n + i] <- -p$s[i]
}
rhs <- rep(c(1, 0), c(p$m, p$n))
obj <- c(t(t(p$c) * p$d), p$f)
dir <- rep(c("==", "<="), c(p$m, p$n))
types <- rep(c("C", "B"), c(p$m, 1) * p$n)

cbind(conditions, dir, rhs)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,dir,rhs
1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,==,1
8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,0,<=,0
0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,<=,0
0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,0,0,<=,0
0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-25000,0,<=,0
0,0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,<=,0


El siguiente paso es añadir la restricción de que únicamente puedan construirse el Almacén A1 o el A3, obteniendo la siguiente matriz de restricciones:

In [12]:
a1 <- rep(0, (1 + p$m) * p$n)
a1[p$m * p$n + c(1, 3)] <- 1
conditions <- rbind(conditions, a1)
dir <- c(dir, "<=")
rhs <- c(rhs, 1)

rownames(conditions) <- NULL
cbind(conditions, dir, rhs)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,dir,rhs
1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,==,1
8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,0,<=,0
0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,<=,0
0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,0,0,<=,0
0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-25000,0,<=,0
0,0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,<=,0


Finalmente, se añadirán las restricciones necesarias para indicar que si el número de libros almacenados en *A4* supera los $22000$, entonces *A1* y *A2* deberán superar $15000$ libros en conjunto. En este caso es necesario añadir una variable binaria adiccional para modelizar la nueva lógica de implicación. La matriz de restricciones se muestra a continuación:

In [13]:
obj <- c(obj, 0)
conditions <- cbind(conditions, rep(0, nrow(conditions)))
types <- c(types, "B")

a2 <- conditions[4, ]
a2[length(a2)] <- -1
conditions <- rbind(conditions, a2)
dir <- c(dir, "<=")
rhs <- c(rhs, 22000 / p$s[4])

a3 <- colSums(conditions[c(1, 2), ])
conditions <- rbind(conditions, a3)
dir <- c(dir, ">=")
rhs <- c(rhs, 15000/sum(p$s[c(1, 2)]))

rownames(conditions) <- NULL
cbind(conditions, dir, rhs)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,dir,rhs
1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,==,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,==,1
8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,0,0,<=,0
0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-20000,0,0,0,0,<=,0
0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,0,0,0,<=,0
0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-25000,0,0,<=,0
0,0,0,0,8000,0,0,0,0,12000,0,0,0,0,9000,0,0,0,0,14000,0,0,0,0,17000,0,0,0,0,-15000,0,<=,0


Por último, resolvemos el problema con la matriz de restricciones indicada, obteniendo la siguiente solución, cuya matriz $Y$ es binaria, a pesar de que esta no se ha restringido de tal manera. Por tanto, esta solución también será la que se obtenga modelizando el problema como de fuente única. Los resultados se muestran a continuación

In [14]:
s <- Solve(obj   = obj, 
           mat   = conditions, 
           dir   = dir,  
           rhs   = rhs,
           types = types)
FormatUncapacitedFacicilityLocationSolution(s, p)

0,0,0,0,0
1,1,0,0,0
0,0,1,0,0
0,0,0,0,1
0,0,0,1,0
0,1,1,1,1
0,0,0,0,0
8000,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


##### WebBook: Cuestión 4

*¿Cómo se debe modificar el modelo de distribución si se exige que cada región de ventas debe ser asignada a un único almacén?*

Para modelizar el problema como de fuente única basta con indicar que todas las variables de decisión deberán ser de tipo binario, lo cual se hace a partir de la siguiente orden:

In [15]:
types.unique <- rep("B", (p$m + 1) * p$n)

Lo siguiente es resolver el problema, que como vemos, proporciona los mismos resultados que el anterior, ya que este ya era de fuente única:

In [16]:
s <- Solve(obj = obj, 
          mat = conditions, 
          dir = dir, 
          rhs = rhs,
          types = types.unique)
FormatUncapacitedFacicilityLocationSolution(s, p)

0,0,0,0,0
1,1,0,0,0
0,0,1,0,0
0,0,0,0,1
0,0,0,1,0
0,1,1,1,1
0,0,0,0,0
8000,12000,0,0,0
0,0,9000,0,0
0,0,0,0,17000
0,0,0,14000,0


## Conclusiones

Los problemas de localización de servicios mediante la programación lineal representan un método de modelización muy enriquecedor, que permite estudiar el problema desde el punto de vista matemático, de manera relativamente sencilla. Estos problemas, presentan una gran complejidad a nivel computacional debido a su carácter de programación binaria, donde la explosión combinatoria hace que para situaciones en que hay que modelizar un gran número de puntos de demanda y abastecimiento, encontrar soluciones en tiempos razonables se convierte en una tarea muy complicada.  

Por dichas razones, la literatura ha propuesto distintas heurísticas que tratan de hacer frente a estas situaciones tratando de proporcionar un contrato razonable entre los resultados óptimos que se obtiene a través de métodos exactos, y el uso reducido de recursos computacionales. Tal y como se ha indicado, las heurísticas greedy más básicas son *ADD* y *DROP*, pero a pesar de ello, proporcionan resultados eficientes. Es por ello que estas técnicas deben ser tenidas en cuenta.

## Referencias

  * [TRC13] Team, R.C., 2013. R: A language and environment for statistical computing.
  * [GP18] Sergio García Prado. Programación Entera: Heurísticas, 2018. [github.com/garciparedes/integer-programming-heuristics](https://github.com/garciparedes/integer-programming-heuristics).
  * [SA18] Jesús Sáez Aguado. Programación Entera, 2017/18. Facultad de Ciencias: Departamento de Estadística e Investigación Operativa.
  * [THBSST17] Theussl, S., Hornik, K., Buchta, C., Schwendinger, F., Schuchardt, H. and Theussl, M.S., 2017. Package ‘Rglpk’.
  